In [1]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

In [9]:
examples = [
    {
        "question": "Dalam pembangunan infrastruktur telekomunikasi, bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) yang digunakan?",
        "answer": """
```sql
SELECT a.article_number, a.text, r.title
FROM articles a
JOIN regulations r ON a.regulation_id = r.id
WHERE a.text ILIKE '%TKDN%' OR a.text ILIKE '%belanja modal%' OR a.text ILIKE '%capital expenditure%'
LIMIT 5;
```
""",
    },
    {
        "question": "Apa isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016?",
        "answer": """
```sql
SELECT a.article_number, a.text
FROM articles a
JOIN regulations r ON a.regulation_id = r.id
WHERE 
r.short_type = 'PERMENKOMINFO' AND 
r.number = 4 AND 
r.YEAR = 2016 AND 
a.article_number = 10
LIMIT 5;
```
""",
    },
#     {
#         "question": "Who was the maternal grandfather of George Washington?",
#         "answer": """
# Are follow up questions needed here: Yes.
# Follow up: Who was the mother of George Washington?
# Intermediate answer: The mother of George Washington was Mary Ball Washington.
# Follow up: Who was the father of Mary Ball Washington?
# Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
# So the final answer is: Joseph Ball
# """,
#     },
    {
        "question": "Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?",
        "answer": """
```sql  
SELECT
    a.article_number,
    a.title,
    a.status,
    r.title AS regulation_title,
    r.short_type,
    r.number,
    r.year
FROM
    articles a
JOIN
    regulations r ON a.regulation_id = r.id
WHERE
    r.short_type = 'PERMENKOMINFO'
    AND r.number = 26
    AND r.year = 2007
    AND a.status = 'ineffective'
```
""",
    },
]

In [10]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?"}).to_string()
)

Question: Dalam pembangunan infrastruktur telekomunikasi, bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) yang digunakan?

```sql
SELECT a.article_number, a.text, r.title
FROM articles a
JOIN regulations r ON a.regulation_id = r.id
WHERE a.text ILIKE '%TKDN%' OR a.text ILIKE '%belanja modal%' OR a.text ILIKE '%capital expenditure%'
LIMIT 5;
```


Question: Apa isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016?

```sql
SELECT a.article_number, a.text
FROM articles a
JOIN regulations r ON a.regulation_id = r.id
WHERE 
r.short_type = 'PERMENKOMINFO' AND 
r.number = 4 AND 
r.YEAR = 2016 AND 
a.article_number = 10
LIMIT 5;
```


Question: Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?

```sql  
SELECT
    a.article_number,
    a.title,
    a.status,
    r.title AS regulation_title,
    r.short_type,
    r.number,
    r.year
FROM
    articles a
JOIN
    regulat

In [11]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import OllamaEmbeddings

embedding_model = OllamaEmbeddings(model="nomic-embed-text")

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embedding_model,
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1,
)

# Select the most similar example to the input.
question = "Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?


answer: 
```sql  
SELECT
    a.article_number,
    a.title,
    a.status,
    r.title AS regulation_title,
    r.short_type,
    r.number,
    r.year
FROM
    articles a
JOIN
    regulations r ON a.regulation_id = r.id
WHERE
    r.short_type = 'PERMENKOMINFO'
    AND r.number = 26
    AND r.year = 2007
    AND a.status = 'ineffective'
```

question: Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?


In [12]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?"}).to_string()
)

Question: Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?

```sql  
SELECT
    a.article_number,
    a.title,
    a.status,
    r.title AS regulation_title,
    r.short_type,
    r.number,
    r.year
FROM
    articles a
JOIN
    regulations r ON a.regulation_id = r.id
WHERE
    r.short_type = 'PERMENKOMINFO'
    AND r.number = 26
    AND r.year = 2007
    AND a.status = 'ineffective'
```


Question: Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?
